In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float, Text 

### Extract CSVs into DataFrames

In [2]:
datosFifa_file = "../Resources/jugadores.csv"
datosFifa_data_df = pd.read_csv(datosFifa_file, encoding='UTF-8')

### Transform DataFrames

In [3]:
datosFifa_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16643 entries, 0 to 16642
Data columns (total 13 columns):
ID                16643 non-null int64
Name              16643 non-null object
Age               16643 non-null int64
Nationality       16643 non-null object
Overall           16643 non-null int64
Potential         16643 non-null int64
Club              16643 non-null object
Value             16643 non-null object
Wage              16643 non-null object
Release_Clause    16643 non-null object
Photo             16643 non-null object
Club_Logo         16643 non-null object
Aggression        16643 non-null int64
dtypes: int64(5), object(8)
memory usage: 1.7+ MB


In [4]:
datosFifa_transformed_df = datosFifa_data_df[['ID','Name','Age','Nationality','Overall',
                                              'Club','Value','Wage','Release_Clause',
                                             'Photo', 'Club_Logo', 'Aggression']].copy()
pv=datosFifa_transformed_df['Value'].to_list()
w=datosFifa_transformed_df['Wage'].to_list()
rc=datosFifa_transformed_df['Release_Clause'].to_list()
pv1=[]
for x in pv:
    try:
        l = len(x) - 1
        euro = x[-l:]
        if euro[-1:] == "M":
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000000
        else:
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000
        pv1.append(number)
    except:
        pv1.append(0)

w1=[]
for x in w:
    try:
        l = len(x) - 1
        euro = x[-l:]
        if euro[-1:] == "M":
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000000
        else:
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000
        w1.append(number)
    except:
        w1.append(0)
    
rc1=[]
for x in rc:
    try:
        l = len(x) - 1
        euro = x[-l:]
        if euro[-1:] == "M":
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000000
        else:
            l = len(euro) - 1
            s = euro[:l]
            number = float(s) * 1000
        rc1.append(number)
    except:
        rc1.append(0)
        
change = {'pv':pv1,'w':w1,'rc':rc1}
clean_values = pd.DataFrame(change)
datosFifa_transformed_df = pd.concat([datosFifa_transformed_df,clean_values], axis=1)
datosFifa_transformed_df = datosFifa_transformed_df[['ID','Name','Age','Nationality','Overall',
                                                     'Club','pv','w','rc',
                                                     'Photo', 'Club_Logo', 'Aggression']]
datosFifa_transformed_df = datosFifa_transformed_df.rename(columns={'ID':'player_id',
                                                                    'Name':'player_name',
                                                                    'Age':'age',
                                                                    'Nationality':'nationality',
                                                                    'Overall':'overall',
                                                                    'Club':'club','pv':'player_value',
                                                                    'w':'wage','rc':'relase_clause',
                                                                    'Photo':'photo','Club_Logo':'club_logo',
                                                                    'Aggression':'aggression'})

In [5]:
datosFifa_transformed_df.head()

,player_id,player_name,age,nationality,overall,club,player_value,wage,relase_clause,photo,club_logo,aggression
0,158023,L. Messi,31,Argentina,94,FC Barcelona,110500000.0,565000.0,226500000.0,https://cdn.sofifa.org/players/4/19/158023.png,https://cdn.sofifa.org/teams/2/light/241.png,48
1,20801,Cristiano Ronaldo,33,Portugal,94,Juventus,77000000.0,405000.0,127100000.0,https://cdn.sofifa.org/players/4/19/20801.png,https://cdn.sofifa.org/teams/2/light/45.png,63
2,190871,Neymar Jr,26,Brazil,92,Paris Saint-Germain,118500000.0,290000.0,228100000.0,https://cdn.sofifa.org/players/4/19/190871.png,https://cdn.sofifa.org/teams/2/light/73.png,56
3,193080,De Gea,27,Spain,91,Manchester United,72000000.0,260000.0,138600000.0,https://cdn.sofifa.org/players/4/19/193080.png,https://cdn.sofifa.org/teams/2/light/11.png,38
4,192985,K. De Bruyne,27,Belgium,91,Manchester City,102000000.0,355000.0,196400000.0,https://cdn.sofifa.org/players/4/19/192985.png,https://cdn.sofifa.org/teams/2/light/10.png,76


In [6]:
datosFifa_transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16643 entries, 0 to 16642
Data columns (total 12 columns):
player_id        16643 non-null int64
player_name      16643 non-null object
age              16643 non-null int64
nationality      16643 non-null object
overall          16643 non-null int64
club             16643 non-null object
player_value     16643 non-null float64
wage             16643 non-null float64
relase_clause    16643 non-null float64
photo            16643 non-null object
club_logo        16643 non-null object
aggression       16643 non-null int64
dtypes: float64(3), int64(4), object(5)
memory usage: 1.5+ MB


## Classes

In [7]:
Base = declarative_base()

In [8]:
class Fifa(Base):
    __tablename__ = 'fifa'
    player_id = Column(Integer, primary_key=True)
    player_name = Column(String(255))
    age = Column(Integer)
    nationality = Column(String(255))
    overall = Column(Integer)
    club = Column(String(255))
    player_value = Column(Float)
    wage = Column(Float)
    relase_clause = Column(Float)
    photo = Column(Text)
    club_logo = Column(Text)
    aggression = Column(Float)


### Database connection

In [9]:
password = '{your password}'
rds_connection_string = f'root:{password}@localhost:3306/fifadb?charset=utf8mb4'

In [10]:
engine = create_engine(f'mysql://{rds_connection_string}')
conn = engine.connect()

In [11]:
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)
session = Session(bind=engine)

## Ad data

In [12]:
df = datosFifa_transformed_df

In [14]:
for index, row in df.iterrows():
    player = Fifa(player_id=row['player_id'], player_name=row['player_name'], age=row['age'], 
                  nationality=row['nationality'], overall=row['overall'],club=row['club'], 
                  player_value=row['player_value'], wage=row['wage'], relase_clause=row['relase_clause'],
                  photo=row['photo'], club_logo=row['club_logo'],
                  aggression=row['aggression'])
    session.add(player)

In [15]:
session.commit()